In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import math
%matplotlib inline
import seaborn as sns
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA

In [2]:
df_x_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
df_x_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
df_y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv').age
df_y_train_mer = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')
IDtrain = pd.DataFrame({'custid': df_x_train.custid.unique()})
IDtest = pd.DataFrame({'custid': df_x_test.custid.unique()})

# df = pd.concat([df_x_train, df_x_test])

eda_df = pd.merge(df_x_train , df_y_train_mer, on = 'custid')

ALL = pd.read_csv(os.path.abspath("../input")+'/ALL피처추가060601.csv')

In [3]:
df = pd.read_csv(os.path.abspath("../input")+'/jin_features.csv')
dft = pd.read_csv(os.path.abspath("../input")+'/jin_features_te.csv')

In [4]:
trainid = IDtrain.custid.to_list()
X_train = ALL.query('custid in @trainid')

testid = IDtest.custid.to_list()
X_test = ALL.query('custid in @testid')

In [5]:
X_train = X_train.drop('custid', axis = 1 )

In [6]:
X_test = X_test.drop('custid', axis = 1 )

In [7]:
numeric_columns2 = X_train.dtypes[X_train.dtypes != 'object'].index.tolist()

In [8]:
# non-skew에만 처리를 하기 위해서 numeric_columns에서 skew와 non-skew로 구분
skewlist2 = []
numeric_columns_ns2 = []

for i in numeric_columns2:
    if 'SKEW' in i:
        skewlist2.append(i)
    else:
        numeric_columns_ns2.append(i)

In [9]:
# non-skew standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[numeric_columns2] = scaler.fit_transform(X_train[numeric_columns2])
X_test[numeric_columns2] = scaler.transform(X_test[numeric_columns2])

### feature concat

In [10]:
data = pd.concat([df, X_train[numeric_columns_ns2], X_train[skewlist2]], axis=1)

X_test.index = dft.index
data_te = pd.concat([dft, X_test[numeric_columns_ns2], X_test[skewlist2] ], axis=1)

In [14]:
display( data, data_te)

,custid,"('tot_amt', '총구매액')","('tot_amt', '구매건수')","('tot_amt', '평균구매가격')","('tot_amt', '최대구매액')","('dis_amt', 'dis_sum')","('dis_amt', 'dis_mean')","('net_amt', 'net_sum')","('net_amt', 'net_mean')","('inst_mon', '평균할부개월수')",...,행사장(여성캐주얼),행사장(여성캐쥬),행사장(잡화),화장품,식품팀,의류패션팀,인터넷백화점_y,잡화가용팀,방문당평균구매액,방문당평균구매상품수
0,0,14.370545,2.484907,11.972655,13.028055,12.067965,9.239061,14.265185,11.867296,1.339774,...,-0.046443,-0.044738,-0.031135,1.514697,-0.346229,-0.451798,-0.006806,0.060923,0.316165,-0.391769
1,2,15.137493,2.484907,12.368329,14.163347,12.449376,9.239061,15.067062,12.330205,1.386294,...,-0.046443,-0.044738,-0.031135,-0.380754,-0.346229,-0.076312,-0.006806,0.556603,1.848414,-0.391769
2,3,14.648637,3.433987,11.247453,13.339088,11.425852,8.024971,14.607979,11.206794,1.299283,...,-0.046443,-0.044738,-0.031135,-0.295355,-0.310157,-0.224343,-0.006806,0.068412,-0.067703,0.702682
3,4,13.859528,1.609438,12.368329,13.235694,9.989711,8.603554,13.838447,12.330205,1.386294,...,-0.046443,-0.044738,-0.031135,-0.559885,-0.346229,-0.118915,-0.006806,-0.559987,1.777211,0.245300
4,5,15.435643,3.496508,11.969913,13.432786,12.797356,9.239061,15.361476,11.895747,1.056053,...,-0.046443,-0.044738,-0.031135,0.404504,0.134538,0.161607,-0.006806,0.736393,0.272373,-0.462554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,16.129623,4.343805,12.278325,14.224968,12.819015,8.942196,16.093831,12.242107,1.035243,...,-0.046443,-0.044738,-0.031135,1.146022,-0.346229,5.361198,-0.006806,0.672847,1.224299,0.169069
21583,29996,14.157184,2.995732,11.212758,12.384223,11.642207,8.697926,14.072863,11.128438,0.793231,...,-0.046443,-0.044738,-0.031135,-0.601543,-0.297154,-0.476080,-0.006806,-0.073786,-0.329461,-0.160108
21584,29997,14.996220,3.044522,12.000494,14.289880,10.604132,7.608871,14.983768,11.988042,0.916291,...,-0.046443,-0.044738,-0.031135,-0.622372,0.078366,-0.422218,-0.006806,0.550462,1.160984,0.988546
21585,29998,13.428231,2.639057,10.863299,11.695255,9.563178,6.999071,13.407047,10.842116,1.172720,...,-0.046443,-0.044738,-0.031135,-0.605709,-0.346229,-0.346326,-0.006806,-0.496227,-0.559442,-0.312135


,custid,"('tot_amt', '총구매액')","('tot_amt', '구매건수')","('tot_amt', '평균구매가격')","('tot_amt', '최대구매액')","('dis_amt', 'dis_sum')","('dis_amt', 'dis_mean')","('net_amt', 'net_sum')","('net_amt', 'net_mean')","('inst_mon', '평균할부개월수')",...,행사장(여성캐주얼),행사장(여성캐쥬),행사장(잡화),화장품,식품팀,의류패션팀,인터넷백화점_y,잡화가용팀,방문당평균구매액,방문당평균구매상품수
0,30001,15.240622,3.332205,11.944791,13.607257,11.679041,8.383425,15.211817,11.915987,1.008228,...,-0.046443,-0.044738,-0.031135,-0.570299,0.266301,-0.017519,-0.006806,0.486510,1.207685,1.285845
1,30002,15.903696,4.543295,11.278736,12.923915,12.660490,8.035838,15.863875,11.238915,1.118030,...,-0.046443,-0.044738,-0.031135,3.468470,-0.140063,1.420423,-0.006806,0.891128,0.473286,2.500668
2,30003,14.147416,3.871201,10.316067,11.835016,9.922849,6.111467,14.132677,10.287882,0.850333,...,-0.046443,-0.044738,-0.031135,-0.464695,-0.253084,-0.165801,-0.006806,-0.393869,-0.714909,0.066920
3,30005,11.790081,1.098612,10.950824,11.141876,8.160804,7.955425,11.746819,10.899532,1.386294,...,-0.046443,-0.044738,-0.031135,-0.566133,-0.346229,-0.553782,-0.006806,-0.561271,-0.708207,-1.241194
4,30007,12.596427,1.945910,10.804685,11.141876,9.220390,7.429125,12.561650,10.769909,0.847298,...,-0.046443,-0.044738,-0.031135,-0.366174,-0.346229,-0.490869,-0.006806,-0.520191,-0.618107,-0.497947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,49988,13.356646,1.609438,11.970357,12.955130,10.360944,8.974745,13.305353,11.919064,1.178655,...,-0.046443,-0.044738,-0.031135,-0.480734,-0.346229,-0.344074,-0.006806,-0.518479,0.674657,0.245300
14376,49990,12.269052,1.098612,12.269052,12.269052,9.273409,9.239061,12.217759,12.217759,1.386294,...,-0.046443,-0.044738,-0.031135,-0.241199,-0.346229,-0.553782,-0.006806,-0.494515,0.124715,-1.241194
14377,49992,12.305923,1.098612,11.612780,12.031725,9.310276,8.617220,12.254629,11.561487,0.693147,...,-0.046443,-0.044738,-0.031135,-0.574465,-0.346229,-0.479611,-0.006806,-0.562982,-0.422558,-1.241194
14378,49993,11.870810,1.609438,10.484536,11.156265,8.160804,6.775366,11.846035,10.459762,0.693147,...,-0.046443,-0.044738,-0.031135,-0.539055,-0.272087,-0.553782,-0.006806,-0.547149,-0.248877,3.218286


In [12]:
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
data.shape, data_te.shape, y_train.shape

((21587, 5191), (14380, 5191), (21587,))

In [13]:
data.columns = data.columns.astype(str)
data_te.columns = data_te.columns.astype(str)

### Modeling

In [15]:
# import re
# data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9]+', ' ', x))
# data_te = data_te.rename(columns = lambda x:re.sub('[^A-Za-z0-9]+', ' ', x))

In [16]:
data2 = data.values
data_te2 = data_te.values

In [17]:
ftr = data2[:, 1:]
target = y_train
target_log = np.log1p(target)

In [18]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.2 , stratify = target , random_state=0)
train_x.shape, valid_x.shape

((17269, 5190), (4318, 5190))

In [19]:
from lightgbm import LGBMRegressor

clf = LGBMRegressor(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric = 'RMSE', 
        verbose=100, early_stopping_rounds= 50)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 7.99213	training's l2: 63.8742	valid_1's rmse: 8.58504	valid_1's l2: 73.703
[200]	training's rmse: 7.25095	training's l2: 52.5763	valid_1's rmse: 8.34465	valid_1's l2: 69.6332
[300]	training's rmse: 6.76285	training's l2: 45.7362	valid_1's rmse: 8.29029	valid_1's l2: 68.7289
[400]	training's rmse: 6.38687	training's l2: 40.7921	valid_1's rmse: 8.27331	valid_1's l2: 68.4476
[500]	training's rmse: 6.06472	training's l2: 36.7808	valid_1's rmse: 8.26928	valid_1's l2: 68.3809
Early stopping, best iteration is:
[457]	training's rmse: 6.20259	training's l2: 38.4721	valid_1's rmse: 8.26796	valid_1's l2: 68.3591


LGBMRegressor(learning_rate=0.02, max_depth=12, n_estimators=1000,
              num_leaves=32, silent=-1, subsample=0.8, verbose=-1)

#### Feature Selection & Cross Validation

In [32]:
ftr = data2[:, 1:]
target = y_train
target_log = np.log1p(target)

In [33]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(clf, threshold='2.0*mean')
smf.fit(ftr, target)
X_new = smf.transform(ftr)
X_te_new = smf.transform(data_te2[:, 1:])
feature_idx = smf.get_support()
# feature_name = ftr.columns[feature_idx]

In [34]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_new, target, scoring='neg_mean_squared_error', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))
print('RMSE:', np.sqrt(-np.mean(scores)))
# mean -
# 1.0mean - 8.21746057149578
# 2.0mean - 8.216354804099296
# 3.0mean - 8.207348627039885
# 3.5mean -       
# 4.0mean - 
# 4.5mean - 
# 5.0mean - 
# 6.0mean - 
# 6.5mean - 
# 7.0mean - 8.220517039736022
# 8.0mean - 
# 9.0mean - 

교차 검증별 정확도: [-67.1745 -68.5738 -69.7623 -65.5635 -66.4682]
평균 검증 정확도: -67.5085
RMSE: 8.216354804099296


In [35]:
X_new.shape, X_te_new.shape

((21587, 972), (14380, 972))

In [60]:
# pd.DataFrame(X_new).to_csv('jin_ha_sel_feature.csv', index = False )
# pd.DataFrame(X_te_new).to_csv('jin_ha_sel_feature_te.csv', index = False )

#### Hyper parameter Tuning_lgbm

In [40]:
# X_new = np.array(data)
# X_te_new = np.array(data_te)

In [36]:
ftr = X_new
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
target = y_train
target_log = np.log1p(target)

In [37]:
target.shape

(21587,)

In [38]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.3, stratify = target,  random_state=1000)
train_x.shape, valid_x.shape

((15110, 972), (6477, 972))

In [39]:
def get_rmse(model):
    pred = model.predict(valid_x)
    mse = mean_squared_error(valid_y , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses



lgbm = LGBMRegressor(random_state = 1000  )
lgbm = lgbm.fit(train_x , train_y)

cat = CatBoostRegressor(random_state=1000 )
cat = cat.fit(train_x , train_y)

xgb = XGBRegressor(random_state = 1000 )
xgb.fit(train_x , train_y )

reg_ridge = Ridge(random_state = 1000)
reg_ridge.fit(train_x, train_y)


models = [lgbm, cat, xgb, reg_ridge ]
get_rmses(models)


# LGBMRegressor 로그 변환된 RMSE: 8.3
# CatBoostRegressor 로그 변환된 RMSE: 8.213
# XGBRegressor 로그 변환된 RMSE: 8.711
# Ridge 로그 변환된 RMSE: 8.399

Learning rate set to 0.063286
0:	learn: 10.2909770	total: 288ms	remaining: 4m 48s
1:	learn: 10.1481013	total: 398ms	remaining: 3m 18s
2:	learn: 10.0221514	total: 507ms	remaining: 2m 48s
3:	learn: 9.9075040	total: 625ms	remaining: 2m 35s
4:	learn: 9.8144400	total: 741ms	remaining: 2m 27s
5:	learn: 9.7153531	total: 852ms	remaining: 2m 21s
6:	learn: 9.6161469	total: 961ms	remaining: 2m 16s
7:	learn: 9.5354290	total: 1.08s	remaining: 2m 14s
8:	learn: 9.4567265	total: 1.2s	remaining: 2m 11s
9:	learn: 9.3902092	total: 1.32s	remaining: 2m 10s
10:	learn: 9.3256660	total: 1.44s	remaining: 2m 9s
11:	learn: 9.2625956	total: 1.56s	remaining: 2m 8s
12:	learn: 9.2099063	total: 1.68s	remaining: 2m 7s
13:	learn: 9.1543947	total: 1.79s	remaining: 2m 6s
14:	learn: 9.1118236	total: 1.91s	remaining: 2m 5s
15:	learn: 9.0664531	total: 2.04s	remaining: 2m 5s
16:	learn: 9.0175304	total: 2.16s	remaining: 2m 5s
17:	learn: 8.9791046	total: 2.29s	remaining: 2m 4s
18:	learn: 8.9413547	total: 2.41s	remaining: 2m 4s

158:	learn: 7.7729158	total: 19.4s	remaining: 1m 42s
159:	learn: 7.7688841	total: 19.5s	remaining: 1m 42s
160:	learn: 7.7652161	total: 19.6s	remaining: 1m 42s
161:	learn: 7.7603348	total: 19.8s	remaining: 1m 42s
162:	learn: 7.7554240	total: 19.9s	remaining: 1m 42s
163:	learn: 7.7510576	total: 20s	remaining: 1m 42s
164:	learn: 7.7471521	total: 20.1s	remaining: 1m 41s
165:	learn: 7.7439962	total: 20.3s	remaining: 1m 41s
166:	learn: 7.7394181	total: 20.4s	remaining: 1m 41s
167:	learn: 7.7360600	total: 20.5s	remaining: 1m 41s
168:	learn: 7.7324076	total: 20.7s	remaining: 1m 41s
169:	learn: 7.7297620	total: 20.8s	remaining: 1m 41s
170:	learn: 7.7271490	total: 20.9s	remaining: 1m 41s
171:	learn: 7.7220814	total: 21s	remaining: 1m 41s
172:	learn: 7.7172570	total: 21.1s	remaining: 1m 41s
173:	learn: 7.7135523	total: 21.3s	remaining: 1m 40s
174:	learn: 7.7101029	total: 21.4s	remaining: 1m 40s
175:	learn: 7.7064795	total: 21.5s	remaining: 1m 40s
176:	learn: 7.7019700	total: 21.6s	remaining: 1m 4

314:	learn: 7.1645222	total: 38.3s	remaining: 1m 23s
315:	learn: 7.1608781	total: 38.4s	remaining: 1m 23s
316:	learn: 7.1562851	total: 38.5s	remaining: 1m 23s
317:	learn: 7.1529613	total: 38.7s	remaining: 1m 22s
318:	learn: 7.1498862	total: 38.8s	remaining: 1m 22s
319:	learn: 7.1464989	total: 38.9s	remaining: 1m 22s
320:	learn: 7.1437533	total: 39s	remaining: 1m 22s
321:	learn: 7.1402518	total: 39.1s	remaining: 1m 22s
322:	learn: 7.1373559	total: 39.3s	remaining: 1m 22s
323:	learn: 7.1333928	total: 39.4s	remaining: 1m 22s
324:	learn: 7.1304405	total: 39.5s	remaining: 1m 22s
325:	learn: 7.1268398	total: 39.7s	remaining: 1m 21s
326:	learn: 7.1236088	total: 39.8s	remaining: 1m 21s
327:	learn: 7.1232240	total: 39.9s	remaining: 1m 21s
328:	learn: 7.1196598	total: 40s	remaining: 1m 21s
329:	learn: 7.1193108	total: 40.1s	remaining: 1m 21s
330:	learn: 7.1162454	total: 40.2s	remaining: 1m 21s
331:	learn: 7.1128066	total: 40.4s	remaining: 1m 21s
332:	learn: 7.1095259	total: 40.5s	remaining: 1m 2

470:	learn: 6.6846079	total: 58.3s	remaining: 1m 5s
471:	learn: 6.6814090	total: 58.4s	remaining: 1m 5s
472:	learn: 6.6785185	total: 58.5s	remaining: 1m 5s
473:	learn: 6.6756706	total: 58.7s	remaining: 1m 5s
474:	learn: 6.6723097	total: 58.8s	remaining: 1m 4s
475:	learn: 6.6704296	total: 58.9s	remaining: 1m 4s
476:	learn: 6.6679033	total: 59s	remaining: 1m 4s
477:	learn: 6.6641493	total: 59.2s	remaining: 1m 4s
478:	learn: 6.6607228	total: 59.3s	remaining: 1m 4s
479:	learn: 6.6585967	total: 59.4s	remaining: 1m 4s
480:	learn: 6.6557399	total: 59.6s	remaining: 1m 4s
481:	learn: 6.6532047	total: 59.7s	remaining: 1m 4s
482:	learn: 6.6495809	total: 59.8s	remaining: 1m 4s
483:	learn: 6.6468179	total: 60s	remaining: 1m 3s
484:	learn: 6.6442837	total: 1m	remaining: 1m 3s
485:	learn: 6.6410888	total: 1m	remaining: 1m 3s
486:	learn: 6.6386595	total: 1m	remaining: 1m 3s
487:	learn: 6.6351921	total: 1m	remaining: 1m 3s
488:	learn: 6.6324010	total: 1m	remaining: 1m 3s
489:	learn: 6.6289673	total: 1m

628:	learn: 6.2361925	total: 1m 16s	remaining: 45.3s
629:	learn: 6.2332698	total: 1m 16s	remaining: 45.2s
630:	learn: 6.2303337	total: 1m 17s	remaining: 45.1s
631:	learn: 6.2265431	total: 1m 17s	remaining: 45s
632:	learn: 6.2243424	total: 1m 17s	remaining: 44.8s
633:	learn: 6.2217792	total: 1m 17s	remaining: 44.7s
634:	learn: 6.2189229	total: 1m 17s	remaining: 44.6s
635:	learn: 6.2160858	total: 1m 17s	remaining: 44.5s
636:	learn: 6.2124884	total: 1m 17s	remaining: 44.4s
637:	learn: 6.2082296	total: 1m 17s	remaining: 44.3s
638:	learn: 6.2054409	total: 1m 18s	remaining: 44.2s
639:	learn: 6.2033462	total: 1m 18s	remaining: 44s
640:	learn: 6.2003467	total: 1m 18s	remaining: 43.9s
641:	learn: 6.2001782	total: 1m 18s	remaining: 43.8s
642:	learn: 6.1976888	total: 1m 18s	remaining: 43.6s
643:	learn: 6.1950842	total: 1m 18s	remaining: 43.5s
644:	learn: 6.1924449	total: 1m 18s	remaining: 43.4s
645:	learn: 6.1896087	total: 1m 18s	remaining: 43.3s
646:	learn: 6.1867202	total: 1m 19s	remaining: 43.

784:	learn: 5.8275165	total: 1m 36s	remaining: 26.5s
785:	learn: 5.8255437	total: 1m 36s	remaining: 26.3s
786:	learn: 5.8216685	total: 1m 36s	remaining: 26.2s
787:	learn: 5.8187036	total: 1m 37s	remaining: 26.1s
788:	learn: 5.8170066	total: 1m 37s	remaining: 26s
789:	learn: 5.8148339	total: 1m 37s	remaining: 25.8s
790:	learn: 5.8125110	total: 1m 37s	remaining: 25.7s
791:	learn: 5.8099953	total: 1m 37s	remaining: 25.6s
792:	learn: 5.8075349	total: 1m 37s	remaining: 25.5s
793:	learn: 5.8043893	total: 1m 37s	remaining: 25.4s
794:	learn: 5.8019313	total: 1m 37s	remaining: 25.2s
795:	learn: 5.7990089	total: 1m 38s	remaining: 25.1s
796:	learn: 5.7964075	total: 1m 38s	remaining: 25s
797:	learn: 5.7940441	total: 1m 38s	remaining: 24.9s
798:	learn: 5.7909128	total: 1m 38s	remaining: 24.8s
799:	learn: 5.7884386	total: 1m 38s	remaining: 24.7s
800:	learn: 5.7865149	total: 1m 38s	remaining: 24.5s
801:	learn: 5.7836114	total: 1m 38s	remaining: 24.4s
802:	learn: 5.7797150	total: 1m 39s	remaining: 24.

940:	learn: 5.4479920	total: 1m 56s	remaining: 7.31s
941:	learn: 5.4454896	total: 1m 56s	remaining: 7.19s
942:	learn: 5.4434202	total: 1m 56s	remaining: 7.07s
943:	learn: 5.4408817	total: 1m 57s	remaining: 6.94s
944:	learn: 5.4388237	total: 1m 57s	remaining: 6.82s
945:	learn: 5.4368575	total: 1m 57s	remaining: 6.69s
946:	learn: 5.4339289	total: 1m 57s	remaining: 6.57s
947:	learn: 5.4319771	total: 1m 57s	remaining: 6.44s
948:	learn: 5.4296886	total: 1m 57s	remaining: 6.32s
949:	learn: 5.4278116	total: 1m 57s	remaining: 6.2s
950:	learn: 5.4248842	total: 1m 57s	remaining: 6.07s
951:	learn: 5.4214456	total: 1m 58s	remaining: 5.95s
952:	learn: 5.4189505	total: 1m 58s	remaining: 5.83s
953:	learn: 5.4166579	total: 1m 58s	remaining: 5.7s
954:	learn: 5.4141506	total: 1m 58s	remaining: 5.58s
955:	learn: 5.4122774	total: 1m 58s	remaining: 5.45s
956:	learn: 5.4100281	total: 1m 58s	remaining: 5.33s
957:	learn: 5.4075416	total: 1m 58s	remaining: 5.21s
958:	learn: 5.4048706	total: 1m 58s	remaining: 5

[8.300104403661345, 8.213310543868028, 8.711355359905992, 8.399157864033706]

In [40]:
from bayes_opt import BayesianOptimization

In [41]:
bayesian_params = {
    'max_depth':(8, 16),
    'num_leaves':(24, 64),
    'min_child_samples':(10, 200),
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1),
    'colsample_bytree':(0.5, 1),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha':(0.01, 50)
}

In [42]:
def lgb_rmse_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree, max_bin, reg_lambda, reg_alpha):
    
    params = {
        "n_estimators":2000, 
        "learning_rate":0.02,
        'max_depth':int(round(max_depth)),
        'num_leaves':int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample':max(min(subsample, 1), 0),
        'colsample_bytree':max(min(colsample_bytree, 1), 0),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    
    lgb_model = LGBMRegressor(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'RMSE', verbose= 100, 
                early_stopping_rounds= 100)
    valid_pred = lgb_model.predict(valid_x)
    RMSE = np.sqrt(mean_squared_error(valid_y, valid_pred))
    
    return RMSE

In [43]:
lgbBO = BayesianOptimization(f=lgb_rmse_eval, pbounds=bayesian_params, random_state=1000)
lgbBO.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.99494	training's l2: 63.919	valid_1's rmse: 8.49855	valid_1's l2: 72.2253
[200]	training's rmse: 7.27637	training's l2: 52.9456	valid_1's rmse: 8.25376	valid_1's l2: 68.1246
[300]	training's rmse: 6.80184	training's l2: 46.2651	valid_1's rmse: 8.19875	valid_1's l2: 67.2196
[400]	training's rmse: 6.42346	training's l2: 41.2608	valid_1's rmse: 8.18056	valid_1's l2: 66.9216
[500]	training's rmse: 6.08938	training's l2: 37.0805	valid_1's rmse: 8.16847	valid_1's l2: 66.7239
[600]	training's rmse: 5.78737	training's l2: 33.4937	valid_1's rmse: 8.16555	valid_1's l2: 66.6763
[700]	training's rmse: 5.51054	training's l2: 30.3661	valid_1's rmse: 8

[400]	training's rmse: 6.24164	training's l2: 38.9581	valid_1's rmse: 8.16844	valid_1's l2: 66.7235
[500]	training's rmse: 5.88321	training's l2: 34.6122	valid_1's rmse: 8.15995	valid_1's l2: 66.5847
[600]	training's rmse: 5.55839	training's l2: 30.8957	valid_1's rmse: 8.15266	valid_1's l2: 66.4659
[700]	training's rmse: 5.26456	training's l2: 27.7156	valid_1's rmse: 8.14984	valid_1's l2: 66.4199
Early stopping, best iteration is:
[651]	training's rmse: 5.40497	training's l2: 29.2137	valid_1's rmse: 8.14907	valid_1's l2: 66.4073
|  8        |  8.149    |  0.7675   |  132.0    |  14.3     |  40.39    |  44.76    |  35.85    |  33.16    |  4.179    |  0.9905   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.58396	training's l2: 57.5165	valid_1's rmse: 8.45274	valid_1's l2: 71.4488
[200]	training's rmse: 6.65393	training's l2: 44.2748	valid_1's rmse: 8.23469	valid_1's l2: 67.8102
[300]	training's rmse: 6.17768	training's l2: 38.1637	valid_1's rmse:

[100]	training's rmse: 7.70776	training's l2: 59.4096	valid_1's rmse: 8.46812	valid_1's l2: 71.709
[200]	training's rmse: 6.82022	training's l2: 46.5154	valid_1's rmse: 8.23814	valid_1's l2: 67.8669
[300]	training's rmse: 6.20141	training's l2: 38.4575	valid_1's rmse: 8.18512	valid_1's l2: 66.9963
[400]	training's rmse: 5.69963	training's l2: 32.4858	valid_1's rmse: 8.16826	valid_1's l2: 66.7205
[500]	training's rmse: 5.25472	training's l2: 27.612	valid_1's rmse: 8.15877	valid_1's l2: 66.5655
[600]	training's rmse: 4.86589	training's l2: 23.6768	valid_1's rmse: 8.15287	valid_1's l2: 66.4694
[700]	training's rmse: 4.5193	training's l2: 20.4241	valid_1's rmse: 8.15326	valid_1's l2: 66.4756
[800]	training's rmse: 4.20697	training's l2: 17.6986	valid_1's rmse: 8.1509	valid_1's l2: 66.4372
Early stopping, best iteration is:
[785]	training's rmse: 4.25227	training's l2: 18.0818	valid_1's rmse: 8.14929	valid_1's l2: 66.4109
|  15       |  8.149    |  0.6659   |  318.9    |  14.7     |  89.29 

[100]	training's rmse: 7.79361	training's l2: 60.7404	valid_1's rmse: 8.47343	valid_1's l2: 71.799
[200]	training's rmse: 6.95953	training's l2: 48.435	valid_1's rmse: 8.23464	valid_1's l2: 67.8092
[300]	training's rmse: 6.38912	training's l2: 40.8209	valid_1's rmse: 8.18224	valid_1's l2: 66.949
[400]	training's rmse: 5.9269	training's l2: 35.1281	valid_1's rmse: 8.15833	valid_1's l2: 66.5583
[500]	training's rmse: 5.51888	training's l2: 30.4581	valid_1's rmse: 8.14835	valid_1's l2: 66.3956
[600]	training's rmse: 5.15822	training's l2: 26.6072	valid_1's rmse: 8.14151	valid_1's l2: 66.2841
[700]	training's rmse: 4.82979	training's l2: 23.3268	valid_1's rmse: 8.13982	valid_1's l2: 66.2567
Early stopping, best iteration is:
[688]	training's rmse: 4.86709	training's l2: 23.6885	valid_1's rmse: 8.13894	valid_1's l2: 66.2424
|  22       |  8.139    |  0.6732   |  294.8    |  15.04    |  117.9    |  26.82    |  46.07    |  13.76    |  3.302    |  0.5363   |
Training until validation scores do

Early stopping, best iteration is:
[703]	training's rmse: 5.05027	training's l2: 25.5052	valid_1's rmse: 8.13667	valid_1's l2: 66.2054
|  28       |  8.137    |  0.5543   |  266.6    |  12.65    |  147.4    |  13.2     |  45.91    |  23.86    |  8.469    |  0.9726   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.84724	training's l2: 61.5792	valid_1's rmse: 8.48472	valid_1's l2: 71.9905
[200]	training's rmse: 7.03253	training's l2: 49.4565	valid_1's rmse: 8.24705	valid_1's l2: 68.0138
[300]	training's rmse: 6.49798	training's l2: 42.2238	valid_1's rmse: 8.19342	valid_1's l2: 67.1322
[400]	training's rmse: 6.09192	training's l2: 37.1115	valid_1's rmse: 8.17342	valid_1's l2: 66.8048
[500]	training's rmse: 5.73866	training's l2: 32.9322	valid_1's rmse: 8.16334	valid_1's l2: 66.6401
[600]	training's rmse: 5.40838	training's l2: 29.2506	valid_1's rmse: 8.15833	valid_1's l2: 66.5584
[700]	training's rmse: 5.10283	training's l2: 26.0388	valid_1's rmse:

In [44]:
lgbBO.res

[{'target': 8.159582609052569,
  'params': {'colsample_bytree': 0.8267947927323047,
   'max_bin': 66.35340213095881,
   'max_depth': 15.602262914792195,
   'min_child_samples': 101.61636627131966,
   'min_child_weight': 43.75125222391973,
   'num_leaves': 32.4933072369088,
   'reg_alpha': 2.045074142206765,
   'reg_lambda': 3.9725474189957124,
   'subsample': 0.61656609867419}},
 {'target': 8.17004539342175,
  'params': {'colsample_bytree': 0.9208703621265308,
   'max_bin': 111.47034874950597,
   'max_depth': 13.93975626865927,
   'min_child_samples': 84.50928428367985,
   'min_child_weight': 9.930569477666833,
   'num_leaves': 53.7415765836856,
   'reg_alpha': 3.488408227688028,
   'reg_lambda': 8.853486706603126,
   'subsample': 0.9763221996107709}},
 {'target': 8.171562888283582,
  'params': {'colsample_bytree': 0.9655717173387555,
   'max_bin': 213.56116699200484,
   'max_depth': 8.23185327509497,
   'min_child_samples': 196.58522207365755,
   'min_child_weight': 17.64224649812914,

In [45]:
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmin(np.array(target_list)))

[8.159582609052569, 8.17004539342175, 8.171562888283582, 8.16914417451538, 8.17904165690679, 8.156273768283857, 8.150212234927787, 8.149067543075125, 8.155718457103376, 8.160374675411536, 8.145628680763515, 8.153547272594029, 8.161748633056996, 8.149915172081283, 8.149286865270724, 8.16491111490235, 8.175468831394106, 8.166836529387334, 8.170130246897218, 8.167038841242782, 8.147513803212206, 8.13894101538311, 8.154683374404343, 8.164748085806893, 8.152783862594912, 8.151197862871083, 8.144653897217102, 8.136670067856032, 8.15429017011101, 8.157789482256456]
maximum target index: 27


In [46]:
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmin(np.array(target_list))]
print(max_dict)

{'target': 8.136670067856032, 'params': {'colsample_bytree': 0.5543446579385094, 'max_bin': 266.60902033120465, 'max_depth': 12.646496395874314, 'min_child_samples': 147.43609706902322, 'min_child_weight': 13.197385976374632, 'num_leaves': 45.91016035048342, 'reg_alpha': 23.860024134242312, 'reg_lambda': 8.46904710281589, 'subsample': 0.9725842699472822}}


In [47]:
ftr = DataFrame(X_new)
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
target = y_train
target_log = np.log1p(target)

In [48]:
from sklearn.model_selection import KFold

def train_apps_all_with_oof(ftr, target, nfolds=5):
    ftr = ftr
    target = target

    # nfolds 개의 cross validatin fold set을 가지는 KFold 생성 
    folds = KFold(n_splits=nfolds, shuffle=True, random_state=0)
    
    # Out of Folds로 학습된 모델의 validation set을 예측하여 결과 확률을 담을 array 생성.
    # validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
    oof_preds = np.zeros((ftr.shape[0],))  
    
    # Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
    test_preds = np.zeros(((X_te_new.shape[0],)))
    
    # n_estimators를 4000까지 확대. 
    clf = LGBMRegressor(
                nthread=4,
                n_estimators=4000,
                learning_rate=0.01,
                max_depth=13,
                num_leaves=37,
                colsample_bytree=0.594,
                subsample=0.713,
                max_bin= 65,
                reg_alpha=8.495,
                reg_lambda=8.393,
                min_child_weight=8,
                min_child_samples=102,
                silent=-1,
                verbose=-1,
                )

    # nfolds 번 cross validation Iteration 반복하면서 OOF 방식으로 학습 및 테스트 데이터 예측
    for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(ftr)):
        print('##### iteration ', fold_idx, ' 시작')
        # 학습용 데이터 세트의 인덱스와 검증용 데이터 세트의 인덱스 추출하여 이를 기반으로 학습/검증 데이터 추출
        train_x = ftr.iloc[train_idx, :]
        train_y = target.iloc[train_idx]
        valid_x = ftr.iloc[valid_idx, :]
        valid_y = target.iloc[valid_idx]
        
        # 추출된 학습/검증 데이터 세트로 모델 학습. early_stopping은 200으로 증가. 
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'RMSE', verbose= 200, 
                early_stopping_rounds= 200)
        # 검증 데이터 세트로 예측된 확률 저장. 사용되지는 않음. 
        #oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)       
        # 학습된 모델로 테스트 데이터 세트에 예측 확률 계산. 
        # nfolds 번 반복 실행하므로 평균 확률을 구하기 위해 개별 수행시 마다 수행 횟수로 나눈 확률을 추후에 더해서 최종 평균 확률 계산. 
        test_preds += clf.predict(X_te_new, num_iteration=clf.best_iteration_)/folds.n_splits
        
        
    return clf, test_preds

In [49]:
clf, test_preds = train_apps_all_with_oof(ftr, target, nfolds=5)

##### iteration  0  시작
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 8.00239	training's l2: 64.0382	valid_1's rmse: 8.56435	valid_1's l2: 73.3482
[400]	training's rmse: 7.31095	training's l2: 53.4501	valid_1's rmse: 8.34053	valid_1's l2: 69.5644
[600]	training's rmse: 6.85789	training's l2: 47.0306	valid_1's rmse: 8.27741	valid_1's l2: 68.5155
[800]	training's rmse: 6.49454	training's l2: 42.179	valid_1's rmse: 8.24908	valid_1's l2: 68.0473
[1000]	training's rmse: 6.1771	training's l2: 38.1565	valid_1's rmse: 8.2356	valid_1's l2: 67.825
[1200]	training's rmse: 5.891	training's l2: 34.7038	valid_1's rmse: 8.22613	valid_1's l2: 67.6693
[1400]	training's rmse: 5.6261	training's l2: 31.653	valid_1's rmse: 8.2199	valid_1's l2: 67.5668
[1600]	training's rmse: 5.37861	training's l2: 28.9295	valid_1's rmse: 8.21931	valid_1's l2: 67.5571
[1800]	

In [50]:
test_preds

array([37.94921532, 42.88769799, 27.90362666, ..., 37.82912055,
       33.07105315, 26.29963301])

In [51]:
IDtest['age'] = test_preds ; sub = IDtest

In [53]:
sub.to_csv('submissions_0613_jin_ha_scaled_lgbm_tun.csv', index=False)